In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from filament.core.grid import Grid
from filament.core.state import State
from filament.core.simulation import Simulation
from filament.test.test_cases import bubble_test, create_results_netcdf, gaussian_test

In [4]:
Lx = 2048
Ly = 1024
Nx = 256
Ny = 128
grid = Grid(Lx, Ly, Nx, Ny)

#T =48*3600 Complete simulation is very long
dt = 3
T = 1*300
Nt = T//dt


cx = 500
cy = 500
radius = 100
wind_norm = 5

ratio_x = Nx//10 # 1 arrow every <ratio> point
ratio_y = Ny//10

In [5]:
def stupid_method(history, verbose, *args, **kwargs):
    last_state = history.state_list[-1]
    
    new_state = State.copy(last_state)
    new_state.t += dt
    print(new_state.t) if verbose else None
    new_state.vars['theta_t'][int(new_state.t) % Nx, int(new_state.t) % Ny] = 1
    history.append(new_state)
    history.pop(0)

In [6]:
methods = [stupid_method]
output_folder = 'output_test'
save_rate = 5
backup_rate = 2
verbose = True

mySim = Simulation(gaussian_test('initial.nc', grid, T, Nt),
                   Lx, Ly, Nx, Ny, T, Nt, 
                   methods, 
                   output_folder, 
                   save_rate, 
                   backup_rate, 
                   verbose)

In [7]:
mySim.run()

backup 0 done
save 0 done
6.0
9.0
backup 2 done
12.0
15.0
backup 4 done
18.0
save 5 done
21.0
backup 6 done
24.0
27.0
backup 8 done
30.0
33.0
backup 10 done
save 10 done
36.0
39.0
backup 12 done
42.0
45.0
backup 14 done
48.0
save 15 done
51.0
backup 16 done
54.0
57.0
backup 18 done
60.0
63.0
backup 20 done
save 20 done
66.0
69.0
backup 22 done
72.0
75.0
backup 24 done
78.0
save 25 done
81.0
backup 26 done
84.0
87.0
backup 28 done
90.0
93.0
backup 30 done
save 30 done
96.0
99.0
backup 32 done
102.0
105.0
backup 34 done
108.0
save 35 done
111.0
backup 36 done
114.0
117.0
backup 38 done
120.0
123.0
backup 40 done
save 40 done
126.0
129.0
backup 42 done
132.0
135.0
backup 44 done
138.0
save 45 done
141.0
backup 46 done
144.0
147.0
backup 48 done
150.0
153.0
backup 50 done
save 50 done
156.0
159.0
backup 52 done
162.0
165.0
backup 54 done
168.0
save 55 done
171.0
backup 56 done
174.0
177.0
backup 58 done
180.0
183.0
backup 60 done
save 60 done
186.0
189.0
backup 62 done
192.0
195.0
backup 6

# Plotting results

In [8]:
from filament.display import animate

In [9]:
variable = 'theta_t'

animate.make_video(output_folder + '/results.nc', output_folder, variable)

/Users/pablorichard/miniconda3/envs/py37/lib/python3.7/site-packages/matplotlib/image.py:397: UserWarning: Warning: converting a masked element to nan.
  dv = (np.float64(self.norm.vmax) -
/Users/pablorichard/miniconda3/envs/py37/lib/python3.7/site-packages/matplotlib/image.py:398: UserWarning: Warning: converting a masked element to nan.
  np.float64(self.norm.vmin))
/Users/pablorichard/miniconda3/envs/py37/lib/python3.7/site-packages/matplotlib/image.py:405: UserWarning: Warning: converting a masked element to nan.
  a_min = np.float64(newmin)
/Users/pablorichard/miniconda3/envs/py37/lib/python3.7/site-packages/matplotlib/image.py:410: UserWarning: Warning: converting a masked element to nan.
  a_max = np.float64(newmax)
<string>:6: UserWarning: Warning: converting a masked element to nan.
/Users/pablorichard/miniconda3/envs/py37/lib/python3.7/site-packages/numpy/ma/core.py:722: UserWarning: Warning: converting a masked element to nan.
  data = np.array(a, copy=False, subok=subok)
